In [ ]:
'''
program to find the nature of dispersion relations around a Weyl point with u = 1 in the BHZ model
parameters are res: resolution of whole BZ to find Weyl points
divisor: how much to divide BZ lengths by when locally investigating Weyl point (e.g. divisor ~2: almost all BZ is investigated, divisor ~100: only tiny fraction)
thetares, rres: polar coordinate resolutions
del_res: resolution of the small region of the BZ we are interested in around the Weyl point
zoom: zoom into the centre of plot

'''

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
from numpy import cos, sin, exp, min, max
from numpy.linalg import eigh
from ipywidgets import interact
import matplotlib
from matplotlib import rcParams
from matplotlib.pyplot import figure
matplotlib.rcParams.update({'font.size': 14})
from scipy.optimize import fsolve

pi = np.pi

def plotter(res = 400, divisor = 10, thetares = 200, rres = 6, del_res = 400, zoom =1): 
    
    u=1
     
    #x[0] is C, [1] and [2] are kx and ky
    
    def det(x): #returns the real and imaginary parts of the determinant of h(k), which we want both to be 0
        u=1           
        return [2 + (u**2-x[0]**2)+2*u*(cos(x[1])+cos(x[2]))+2*cos(x[1])*cos(x[2]),
                u + cos(x[1])+cos(x[2]),
                0]

    def findweyl():
        u_weyl=1
        root = fsolve(det, [15, pi/2, pi]) #looks for solutions where det(h(k))=0
        print(root)
        print(np.isclose(det(root), [0.0, 0.0, 0.0]))
        return root
     
    weyl = findweyl()    #weyl object here is a Weyl point it found
    CC = weyl[0]   
    kx_crit = weyl[1] #crit refers to the k_x and k_y value where the Weyl point is exactly
    ky_crit = weyl[2]
    
    
    
    
    
    kxvals = np.linspace(-pi/divisor, pi/divisor, del_res, endpoint=False) #divisor refers to the fraction of the BZ we are interested in
    kyvals = np.linspace(-pi/divisor, pi/divisor, del_res, endpoint=False)
    ky, kx = np.meshgrid((kx_crit + kxvals), (ky_crit + kyvals)) #meshgrid around the Weyl point we found

    #build up a Hamiltonian around the Weyl point
    HH = np.zeros((del_res, del_res, 4, 4), dtype=complex)
    HH[:, :, 0, 0] = u+cos(kx)+cos(ky)
    HH[:, :, 0, 1] = sin(kx)-1.j*sin(ky)
    HH[:, :, 1, 0] =  HH[:, :, 0, 1].conj() 
    HH[:, :, 1, 1] = -HH[:, :, 0, 0]
    HH[:, :, 2, 2] = u+cos(kx)+cos(ky)
    HH[:, :, 2, 3] = -sin(kx)-1.j*sin(ky)
    HH[:, :, 3, 2] =  HH[:, :, 2, 3].conj() 
    HH[:, :, 3, 3] = -HH[:, :, 2, 2]
    HH[:, :, 0, 3] = -CC
    HH[:, :, 1, 2] = -HH[:, :, 0, 3] 
    HH[:, :, 2, 1] = HH[:, :, 1, 2].conj()
    HH[:, :, 3, 0] = HH[:, :, 0, 3].conj()
    
    es, vects = np.linalg.eigh(HH)   
   
    thetavals = np.linspace(-pi/zoom, pi/zoom, thetares)
    rvals = np.linspace(0, pi/divisor, rres) #thetavals and rvals will be used later to plot the bandgap in these polar coordinates
    
    gap = np.zeros([thetares, rres])
    
    
    def find_nearest(array, value): #returns the position of array that's elements is closest to the value
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin();
        return idx;
    
    thetano = -1
    for theta in thetavals:
        thetano+=1
        
        rno = -1
        for r in rvals:
            rno+=1
            kx = cos(theta)*r
            ky = sin(theta)*r #converts polars into Cartesians for simplicity
            
            #index of kx, ky with these values:
            kxno = find_nearest(kxvals, kx)
            kyno = find_nearest(kyvals, ky)
                        
            gap[thetano, rno] = (es[kxno, kyno, 2]-es[kxno, kyno, 1]) #fills up gap array2d with differences of the 2nd and 3rd band 
    parabola = []
    
    
    figure(num=None, figsize=(8, 6), dpi=80)

    for r in range(rres):
        plt.scatter(thetavals, gap[:, r], label = str(np.round(rvals[r], 3)), s= 1)
        parabola.append(min(gap[:, r]))
    plt.legend()
    x_label = "Polar angle " + r"$\theta$"
    plt.xlabel(x_label) 
    plt.title("Bandgaps as a function of polar angle for multiple radii")
    plt.ylabel("Bandgap")
    plt.savefig("bandgap-theta.pdf")
    plt.show()
    z = np.polyfit(range(rres), parabola, 5)
    plt.scatter(np.log(range(rres)), np.log(parabola))
    plt.grid()
    plt.title("Log-log plot of bandgap vs radius") #in the direction perpendicular to linear spacing
    plt.xlabel("log of radius in reciprocal space (arbitrary units)")
    plt.ylabel("log of bandgap")
    plt.savefig("loglog.pdf")
    print(z)
    plt.show()
    
interact(plotter, res = (2, 3000, 100), divisor = (2, 140, 4), thetares = (10, 1200, 10), rres = (4, 60, 4), del_res=(20, 10000, 20), zoom=(1, 40, 1))




interactive(children=(IntSlider(value=400, description='res', max=3000, min=2, step=100), IntSlider(value=10, …

<function __main__.plotter(res=400, divisor=10, thetares=200, rres=6, del_res=400, zoom=1)>